[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/examples-gallery/blob/main/tabular-classification/sklearn/fraud-detection/fraud-classifier-sklearn.ipynb)


# <a id="top">Fraud classification using sklearn</a>

This notebook illustrates how sklearn models can be uploaded to the Openlayer platform.

## <a id="toc">Table of contents</a>

1. [**Getting the data and training the model**](#1)
    - [Downloading the dataset](#download)
    - [Preparing the data](#prepare)
    - [Training the model](#train)
    

2. [**Using Openlayer's Python API**](#2)
    - [Instantiating the client](#client)
    - [Creating a project](#project)
    - [Uploading datasets](#dataset)
    - [Uploading models](#model)
    - [Committing and pushing to the platform](#commit)

In [ ]:
%%bash

if [ ! -e "requirements.txt" ]; then
    curl "https://raw.githubusercontent.com/openlayer-ai/examples-gallery/main/tabular-classification/sklearn/fraud-detection/requirements.txt" --output "requirements.txt"
fi

In [ ]:
!pip install -r requirements.txt

## <a id="1"> 1. Getting the data and training the model </a>

[Back to top](#top)

In this first part, we will get the dataset, pre-process it, split it into training and validation sets, and train a model. Feel free to skim through this section if you are already comfortable with how these steps look for an sklearn model.   

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

### <a id="download">Downloading the dataset </a>


We have stored a sample of the original dataset on the following S3 bucket. If, for some reason, you get an error reading the csv directly from it, feel free to copy and paste the URL in your browser and download the csv file. Alternatively, you can also find the full dataset on [this Kaggle competition](https://www.kaggle.com/datasets/kartik2112/fraud-detection?select=fraudTrain.csv). The dataset in our example corresponds to the first 10,000 rows of the original Kaggle competition dataset.

In [ ]:
DATASET_URL = "https://openlayer-static-assets.s3.us-west-2.amazonaws.com/examples-datasets/tabular-classification/fraudTrainSample.csv"

In [ ]:
data = pd.read_csv(DATASET_URL)

In [ ]:
# Relevant columns
feature_names = ['amt', 'cc_num', 'merchant', 'category','state','job']
label = ['is_fraud']

# Outputs
class_names = ["normal", "fraudulent"]

clean_raw_data = data[feature_names + label]

In [ ]:
X = clean_raw_data.drop('is_fraud', 1)
y = clean_raw_data['is_fraud']

In [ ]:
X.head()

### <a id="prepare">Preparing the data</a>

In [ ]:
def data_encode_one_hot(df, encoders):
    """ Encodes categorical features using one-hot encoding. """
    df = df.copy(True)
    df.reset_index(drop=True, inplace=True) # Causes NaNs otherwise
    enc_dfs = []
    for feature, enc in encoders.items():
        enc_df = pd.DataFrame(enc.transform(df[[feature]]).toarray(), columns=enc.get_feature_names([feature]))
        enc_dfs.append(enc_df)
    df = pd.concat([df] + enc_dfs, axis=1)
    df.drop(list(encoders.keys()), axis=1, inplace=True)
    return df

In [ ]:
def create_encoder_dict(df, categorical_feature_names):
    """ Creates encoders for each of the categorical features. 
        The predict function will need these encoders. 
    """
    from sklearn.preprocessing import OneHotEncoder
    encoders = {}
    for feature in categorical_feature_names:
        enc = OneHotEncoder(handle_unknown='error')
        enc.fit(df[[feature]])
        encoders[feature] = enc
    return encoders

In [ ]:
categorical_feature_names = ['cc_num', 'merchant', 'category', 'state', 'job']

In [ ]:
encoders = create_encoder_dict(X, categorical_feature_names)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_train_one_hot = data_encode_one_hot(x_train, encoders)
x_val_one_hot = data_encode_one_hot(x_val, encoders)

x_val_one_hot

### <a id="train">Training the model</a>

In [ ]:
sklearn_model = GradientBoostingClassifier(random_state=1300)
sklearn_model.fit(x_train_one_hot, y_train)

In [ ]:
print(classification_report(y_val, sklearn_model.predict(x_val_one_hot)))

## <a id="2"> 2. Using Openlayer's Python API</a>

[Back to top](#top)

Now it's time to upload the datasets and model to the Openlayer platform.

In [ ]:
!pip install openlayer

### <a id="client">Instantiating the client</a>

In [ ]:
import openlayer

client = openlayer.OpenlayerClient("YOUR_API_KEY_HERE")

### <a id="project">Creating a project on the platform</a>

In [ ]:
from openlayer.tasks import TaskType

project = client.create_or_load_project(name="Fraud classification", 
                                        task_type=TaskType.TabularClassification,
                                        description="Evaluation of ML approaches to detect frauds")

### <a id="dataset">Uploading datasets</a>

In [ ]:
# Add the ground truths to the ordinal dataset for Openlayer
x_val['is_fraud'] = y_val.values
x_train['is_fraud'] = y_train.values

In [ ]:
from openlayer.datasets import DatasetType

# Validation set
project.add_dataframe(
    df=x_val.sample(1000),
    dataset_type=DatasetType.Validation,
    class_names=class_names,
    label_column_name='is_fraud',
    feature_names=feature_names,
    categorical_feature_names=categorical_feature_names,
)

# Training set
project.add_dataframe(
    df=x_train.sample(1000),
    dataset_type=DatasetType.Training,
    class_names=class_names,
    label_column_name='is_fraud',
    feature_names=feature_names,
    categorical_feature_names=categorical_feature_names,
)

We can check that both datasets are now staged using the `project.status()` method. 

In [ ]:
project.status()

### <a id="model">Uploading models</a>

To upload a model to Openlayer, you will need to create a model package, which is nothing more than a folder with all the necessary information to run inference with the model. The package should include the following:
1. A `requirements.txt` file listing the dependencies for the model.
2. Serialized model files, such as model weights, encoders, etc., in a format specific to the framework used for training (e.g. `.pkl` for sklearn, `.pb` for TensorFlow, and so on.)
3. A `prediction_interface.py` file that acts as a wrapper for the model and implements the `predict_proba` function. 
4. A `model_config.yaml` file that provides information about the model to the Openlayer platform, such as the framework used, feature names, and categorical feature names.

Lets prepare the model package one piece at a time

In [ ]:
# Creating the model package folder (we'll call it `model_package`)
!mkdir model_package

**1. Adding the `requirements.txt` to the model package**

In [ ]:
!scp requirements.txt model_package

**2. Serializing the model and other objects needed**

In [ ]:
import pickle 

# Trained model
with open('model_package/model.pkl', 'wb') as handle:
    pickle.dump(sklearn_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Encoder for the categorical features
with open('model_package/encoders.pkl', 'wb') as handle:
    pickle.dump(encoders, handle, protocol=pickle.HIGHEST_PROTOCOL)

**3. Writing the `prediction_interface.py` file**

In [ ]:
%%writefile model_package/prediction_interface.py

import pickle
from pathlib import Path

import pandas as pd
from sklearn.preprocessing import OneHotEncoder

PACKAGE_PATH = Path(__file__).parent


class SklearnModel:
    def __init__(self):
        """This is where the serialized objects needed should
        be loaded as class attributes."""

        with open(PACKAGE_PATH / "model.pkl", "rb") as model_file:
            self.model = pickle.load(model_file)
        with open(PACKAGE_PATH / "encoders.pkl", "rb") as encoders_file:
            self.encoders = pickle.load(encoders_file)

    def _data_encode_one_hot(self, df: pd.DataFrame) -> pd.DataFrame:
        """Pre-processing needed for our particular use case."""

        df = df.copy(True)
        df.reset_index(drop=True, inplace=True)  # Causes NaNs otherwise
        for feature, enc in self.encoders.items():
            enc_df = pd.DataFrame(
                enc.transform(df[[feature]]).toarray(),
                columns=enc.get_feature_names([feature]),
            )
            df = df.join(enc_df)
            df = df.drop(columns=feature)
        return df

    def predict_proba(self, input_data_df: pd.DataFrame):
        """Makes predictions with the model. Returns the class probabilities."""

        encoded_df = self._data_encode_one_hot(input_data_df)
        return self.model.predict_proba(encoded_df)


def load_model():
    """Function that returns the wrapped model object."""
    return SklearnModel()

**4. Creating the `model_config.yaml`**

In [ ]:
import yaml 

model_config = {
    "name": "Fraud detection model",
    "model_type": "sklearn",
    "class_names": class_names,
    "categorical_feature_names": categorical_feature_names,
    "feature_names":feature_names
}

with open('model_package/model_config.yaml', 'w') as model_config_file:
    yaml.dump(model_config, model_config_file, default_flow_style=False)

Lets check that the model package contains everything needed:

In [ ]:
test_ = x_val.loc[:, x_val.columns != 'is_fraud']

In [ ]:
from openlayer.validators import ModelValidator

model_validator = ModelValidator(
    model_package_dir="model_package", 
    sample_data = test_.iloc[:10, :]
)
model_validator.validate()

Now, we are ready to add the model:

In [ ]:
project.add_model(
    model_package_dir="model_package",
    sample_data=test_.iloc[:10, :]
)

We can check that both datasets and model are staged using the `project.status()` method.

In [ ]:
project.status()

### <a id="commit"> Committing and pushing to the platform </a>

Finally, we can commit the first project version to the platform. 

In [ ]:
project.commit("Initial commit!")

In [ ]:
project.status()

In [ ]:
project.push()